In [1]:
import pandas as pd

# Collect data collection from all partners
   
- P1 - Tübingen
- P2 - Montreal
- P3 - Nijmegen
- P4 - Pisa
- P6 - Istanbul
- P7 - Paris
- P8 - Cambridge



## SACS

In [4]:
# clean variantid naming for p7
p7_sacs["Variant ID \n(consecutive number)"] = p7_sacs["Variant ID \n(consecutive number)"].astype(str)
for idx, row in p7_sacs.iterrows():
    before = row['Variant ID \n(consecutive number)']
    p7_sacs.at[idx, 'Variant ID \n(consecutive number)'] = "P7_SACS_Paris_" + str(before)
    
def get_variant_id(df, l_idx):
    return ",".join([row["Variant ID \n(consecutive number)"] for idx, row in df.iterrows() if idx in l_idx])


all_sampl = [p7_sacs[p7_sacs["local subject ID"]==sample] for sample in p7_sacs["local subject ID"].unique()]
for subdf in all_sampl:
    for idx, row in subdf.iterrows():
        all_idx = [idx for idx, row in subdf.iterrows() if row['Het / Hom']]
        if len(subdf.index)==1:
            p7_sacs.at[idx, "Compound het with Variant ID\n(enter variant ID, 'N/A' or 'unknown')"] = ""
        elif row['Het / Hom']=="hom":
            p7_sacs.at[idx, "Compound het with Variant ID\n(enter variant ID, 'N/A' or 'unknown')"] = ""
        else:
            all_idx.remove(idx)
            p7_sacs.at[idx, "Compound het with Variant ID\n(enter variant ID, 'N/A' or 'unknown')"] = get_variant_id(subdf, all_idx)
            
        
        


NameError: name 'p7_sacs' is not defined

In [17]:
p1_sacs = pd.read_csv("SACS/P1_tubingen/P1_SACS_3.csv", sep="\t")
p2_sacs = pd.read_csv("SACS/P2_montreal/P2_Montreal_variant_collection.SACS.tsv", sep="\t")
p3_sacs = pd.read_csv("SACS/P3_nijmegen/P3_Nijmegen_variant_collection.SACS.final.csv", sep="\t")
p4_sacs = pd.read_csv("SACS/P4_pisa/P4_SACS_variant_collection.JUNE_cleaned_5.final.tsv", sep="\t")
p6_sacs = pd.read_csv("SACS/P6_istanbul/P6_SACS_collection.final_4.txt", sep="\t")
p7_sacs = pd.read_csv("SACS/P7_paris/P7_SACS_data_collection_Paris_v3.csv", sep="\t")


In [18]:
# unify column names
for table in [p2_sacs, p3_sacs, p4_sacs, p6_sacs, p7_sacs]:
    table.columns = p1_sacs.columns
    

In [19]:
# concat all SACS tables
all_table = p1_sacs.append([p2_sacs, p3_sacs, p4_sacs, p6_sacs, p7_sacs])

In [20]:
# unify columns content
def clean_hom_het(x):
    if x in ["homozygous", "HOM", "hom"]:
        x = "Hom"
    elif x in ["heterozygous", "het"]:
        x = "Het"
    else:
         pass
    return x

all_table['Het / Hom'] = all_table['Het / Hom'].apply(lambda x: clean_hom_het(x))

# all_table["Chromosome\n(genome build hg19 / GRCh37)"] = all_table["Chromosome\n(genome build hg19 / GRCh37)"].astype(int)


In [21]:
# number of unique sample
len(all_table["local subject ID"].unique())

379

In [22]:
len(all_table["Protein position"].unique())

307

In [23]:
all_table["Variant_based_id"] = all_table[["Chromosome\n(genome build hg19 / GRCh37)", "Position\n(use vcf format)", "Reference allele", "Alternative allele"]].apply(lambda x: "-".join(x.values.astype(str)), axis=1)

In [24]:
len(all_table["Variant_based_id"].unique())

409

In [25]:
all_table.to_csv("SACS/SACS_all_partners.tsv", sep="\t", index=False)

In [30]:
all_table = pd.read_csv("SACS/SACS_all_partners.finaledit.csv", sep="\t")

In [31]:
import numpy as np

groups_variant = all_table.groupby(["Variant_based_id"])

dict_v_id_count = {}
for i in range(len(groups_variant.size())):
    var = groups_variant.size().index[i]
    count = groups_variant.size()[i]
    dict_v_id_count[var] = count

with open("header_custom_vcf", "w") as f:
    f.write("##fileformat=VCFv4.2\n")
    f.write('##INFO=<ID=NBCOUNT,Number=1,Type=Integer,Description="Number of Samples With Data">\n')
    f.write('##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n')
    f.write("##contig=<ID=13,length=115169878>\n")
    f.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tSAMPLE\n')
with open("SACS_all_prospax_var_vcf", "w") as f:
    for v in dict_v_id_count:
        v_def = v.split("-")
        chrom = v_def[0]
        pos = v_def[1]
        ide = v
        ref = v_def[2]
        alt = v_def[3]
        qual = 30
        filt = "PASS"
        info = "NBCOUNT="+str(dict_v_id_count[v])
        frmt = "GT"
        res = "1/1"
        
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(chrom, pos, ide, ref, alt, qual, filt, info, frmt, res))
        

In [12]:
from cyvcf2 import VCF

vcf_sacs = VCF("SACS/SACS_all_prospax.annotated.vcf")


sacs_hgvsp = {}
for variant in vcf_sacs:
    vep_annot = variant.INFO.get("CSQ")
    for tr in vep_annot.split(","):
        tr_annot = tr.split("|")
        if tr_annot[6]=="ENST00000382298.3":
            prot_desc = tr_annot[11]
            if prot_desc!="":
                csq = tr_annot[1]
                sacs_hgvsp[prot_desc.replace("%3D", "=")] = csq
                
            
            
        
    

In [13]:
with open("SACS/sacs_all_hgvsp.txt", "w") as f:
    f.write("Hugo_Symbol\tProtein_Change\tMutation_Type\n")
    for p in sacs_hgvsp:
        p_clean = p.split(":")[1]
        f.write("SACS\t{}\t{}\n".format(p_clean, sacs_hgvsp[p]))

# SPG7

In [2]:
p1_spg7 = pd.read_csv("SPG7/P1_tubingen/P1_SPG7_2.csv", sep="\t")
p2_spg7 = pd.read_csv("SPG7/P2_montreal/P2_Montreal_variant_collection.SPG7", sep="\t")
p3_spg7 = pd.read_csv("SPG7/P3_nijmegen/P3_Nijmegen_variant_collection.SPG7.final.csv", sep="\t")
p4_spg7 = pd.read_csv("SPG7/P4_pisa/P4_SPG7_variant_collection.JUNE_cleaned_5.final.tsv", sep="\t")
p6_spg7 = pd.read_csv("SPG7/P6_istanbul/P6_SPG7_collection.final_4.txt", sep="\t")
p7_spg7 = pd.read_csv("SPG7/P7_paris/P6_SPG7_data_collection_Paris_final.csv", sep="\t")


In [3]:
# unify column names
for table in [p2_spg7, p3_spg7, p4_spg7, p6_spg7, p7_spg7]:
    table.columns = p1_spg7.columns
    
    

In [4]:
# concat all SPG7 tables
all_table_spg7 = p1_spg7.append([p2_spg7, p3_spg7, p4_spg7, p6_spg7, p7_spg7])

In [5]:
all_table_spg7["Het / Hom"].unique()

array(['homozygous', 'heterozygous', 'Hom', 'Het', 'HOM'], dtype=object)

In [6]:
# unify columns content
def clean_hom_het(x):
    if x in ["homozygous", "HOM", "hom"]:
        x = "Hom"
    elif x in ["heterozygous", "het"]:
        x = "Het"
    else:
         pass
    return x

all_table_spg7['Het / Hom'] = all_table_spg7['Het / Hom'].apply(lambda x: clean_hom_het(x))

# all_table["Chromosome\n(genome build hg19 / GRCh37)"] = all_table["Chromosome\n(genome build hg19 / GRCh37)"].astype(int)


In [7]:
all_table_spg7.columns

Index(['Submitting institution\n(select from dropdown)', 'local subject ID',
       'local family ID', 'PROSPAX subject ID',
       'NGS database ID (e.g. GENESIS, GPAP, etc)',
       'Main phenotype \n(select from dropdown)',
       'Case solved? \n(solved SPG7 / solved other gene / unsure / not solved)',
       'Variant ID \n(consecutive number)', 'Gene',
       'Chromosome\n(genome build hg19 / GRCh37)',
       'Position\n(use vcf format)', 'Reference allele', 'Alternative allele',
       'Transcript\n(use reference transcript ENST00000268704.2)',
       'cDNA change (e.g. ', 'Protein position', 'Het / Hom',
       'Compound heterozygous with Variant ID\n(enter variant ID, 'N/A' or 'unknown')',
       'PaxGene available?\n(yes / no / unknown)',
       'PBMC available?\n(yes / no / unknown)',
       'Fibroblasts available?\n(yes / no / unknown)', 'Comment'],
      dtype='object')

In [8]:
    
all_table_spg7["Variant_based_id"] = all_table_spg7[['Chromosome\n(genome build hg19 / GRCh37)', 'Position\n(use vcf format)', "Reference allele", "Alternative allele"]].apply(lambda x: "-".join(x.values.astype(str)), axis=1)

In [9]:
len(all_table_spg7["Variant_based_id"].unique())

316

In [10]:
all_table_spg7.to_csv("SPG7/SPG7_all_partners.tsv", sep="\t", index=False)

In [121]:
import numpy as np

groups_variant_spg7 = all_table_spg7.groupby(["Variant_based_id"])

dict_v_id_count = {}
for i in range(len(groups_variant_spg7.size())):
    var = groups_variant_spg7.size().index[i]
    count = groups_variant_spg7.size()[i]
    dict_v_id_count[var] = count

with open("header_custom_vcf", "w") as f:
    f.write("##fileformat=VCFv4.2\n")
    f.write('##INFO=<ID=NBCOUNT,Number=1,Type=Integer,Description="Number of Samples With Data">\n')
    f.write('##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n')
    
with open("SPG7_all_prospax_var_vcf", "w") as f:
    for v in dict_v_id_count:
        v_def = v.split("-")
        chrom = v_def[0]
        pos = v_def[1]
        ide = v
        ref = v_def[2]
        alt = v_def[3]
        qual = 30
        filt = "PASS"
        info = "NBCOUNT="+str(dict_v_id_count[v])
        frmt = "GT"
        res = "./."
        
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(chrom, pos, ide, ref, alt, qual, filt, info, frmt, res))
        

In [1]:
from cyvcf2 import VCF

vcf_spg7 = VCF("SPG7/SPG7_all_prospax.annotated.vcf")


spg7_hgvsp = {}
for variant in vcf_spg7:
    vep_annot = variant.INFO.get("CSQ")
    for tr in vep_annot.split(","):
        tr_annot = tr.split("|")
        if tr_annot[6]=="ENST00000268704.2":
            prot_desc = tr_annot[11]
            if prot_desc!="":
                csq = tr_annot[1]
                spg7_hgvsp[prot_desc.replace("%3D", "=")] = csq
                
            

In [2]:
with open("SPG7/spg7_all_hgvsp.txt", "w") as f:
    f.write("Hugo_Symbol\tProtein_Change\tMutation_Type\n")
    for p in spg7_hgvsp:
        p_clean = p.split(":")[1]
        f.write("SPG7\t{}\t{}\n".format(p_clean, spg7_hgvsp[p]))

In [14]:
variable_1 = 23

variable_2 = "chr1"


print(isinstance(variable_2, int))

False
